In [4]:
import os
from pathlib import Path
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, TextLoader

In [5]:
CONFIG = {
    "data_path": "dulieu_daotao",
    "vectorstore_path": "faiss_index",
    "embedding_model": "sentence-transformers/all-MiniLM-L6-v2",
    "retriever_k": 3
}

In [7]:
# Giả sử code tạm phần data của a Quân

def prepare_vector_database(config):
    vectorstore_path = Path(config["vectorstore_path"])
    data_path = Path(config["data_path"])

    if (vectorstore_path.exists()):
        print("csdl vector đã tồn tại. Bỏ qua tạo mới")
        return
    
    if not data_path.exists():
        data_path.mkdir(parents=True, exist_ok=True)
        (data_path / "hoc_phi.txt").write_text("Quy định về học phí năm học 2024-2025 cho sinh viên ngành Công nghệ thông tin là 30 triệu đồng/năm. Sinh viên có thể đóng theo hai kỳ. Ngành Quản trị kinh doanh có mức học phí là 28 triệu đồng/năm.", encoding='utf-8')
        (data_path / "lich_nghi.txt").write_text("Trường thông báo lịch nghỉ lễ Quốc Khánh. Toàn thể sinh viên được nghỉ vào ngày 2 tháng 9. Lịch thi cuối kỳ sẽ được thông báo sau ngày nghỉ lễ.", encoding='utf-8')
        (data_path / "thu_tuc.txt").write_text("Thủ tục nhập học cho tân sinh viên yêu cầu nộp bản sao công chứng học bạ THPT và giấy báo trúng tuyển trước ngày 15 tháng 8. Thủ tục đăng ký thi lại cần hoàn thành đơn trực tuyến trên cổng thông tin.", encoding='utf-8')

    # Tải dữ liệu
    loader = DirectoryLoader(
        str(data_path),
        glob="**/*.txt",
        loader_cls=TextLoader,
        loader_kwargs={'encoding': 'utf-8'}
    )
    documents = loader.load()

    #Chia chunk
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 500,
        chunk_overlap = 50
    )
    docs = text_splitter.split_documents(documents)
    print("Đã chia {len(documents)} tài liệu thành {len(docs)} chunks")

    embeddings_model = HuggingFaceEmbeddings(
        model_name = config["embedding_model"],
        model_kwargs = {'device' : 'cpu'}
    )

    vectorstore = FAISS.from_documents(docs, embeddings_model)
    vectorstore.save.local(config["vectorestore_path"])
